# Aufbereitung der Daten

Damit die Spektogramme überhaupt von Modellen genutzt werden können, müssen sie aufbereitet, (teilweise) bearbeitet und in einem gut verwendbarem Format abgespeichert werden.
In unserem Fall besteht das aus den folgenden Schritten:
1. Die Skala/Rahmen der Bilder entfernen
2. Die Bilder runterskalieren
3. (Optional) Filter oder andere Bildbearbeitungen anwenden
4. Daten nach Klasse aussortieren
5. 

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import json
from tqdm.auto import tqdm

#### Things tried:
- fastNlMeansDenoising on big and small image ~ 83/80%
- grayscale ~ 78%
- grayscale + fastNlMeansDenoising on big and small image ~ 76%
- bilateral Filter ~ 82%
- bilateral filter + contrast(1.7) + brightness(-100) ~ 90% (max; avg: 89%) | avg90,best91
- bilateral filter + contrast(1.7) + brightness(-150) ~ a:90%b:91%
- contrast(1.7) + brightness(-100) ~ a:89%b:90%
- contrast(1.7) + brightness(-150) ~ a:89%b:90%
- contrast(1.5) + brightness(-150) ~ a:88%b:89%
- contrast(2) + brightness(-150) ~ a:%b:%
- hist equ + c/b (1.7/-150) + bil ~ a:82%b:82%
- filter2d (-1/7)+ bil + nl + bil + c1.7/b-150 ~ a:91%b:91%
- f2d (sobel) + bil + nl + bil + c1.7/b-150 ~ a:89%b:91%
- filter2d (outline)+ bil + nl + c1.7/b-150 + f2d (sobel) ~a:88%b:88%
- filter2d (-1/7)+ bil + nl + bil + c1.7/b-150 + sat=255 ~ a:91%b:92% aber stark schwankend


In [2]:
folder_path = Path("../Bat_Orientation_Calls")

def remove_black_frame(img, frame):
    return img[frame[0]:frame[1], frame[2]:frame[3]]

image_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path) if file_name.endswith(".png")]

for image_path in tqdm(image_paths):
    img = cv2.imread(image_path, -1)
    
    #denoise
    #img = cv2.fastNlMeansDenoising(img, None, h=15, templateWindowSize=7, searchWindowSize=21)

    # remove outer frame
    frame = (36, 251, 55, 388)
    img = remove_black_frame(img, frame)
    
    base_width = 128 # might change, since 
    height, width, channels = img.shape
    aspect_ratio = width / height  # Width / Height

    new_width = base_width
    new_height = int(new_width / aspect_ratio)

    img = cv2.resize(img, (new_width, new_height)) 
    # into grayscale
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # increase contrast and decrease brightness
    #img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    #img = cv2.bilateralFilter(img, 9, 75, 75)
    #img = cv2.addWeighted(img, 1.7, np.zeros(img.shape, img.dtype), 0, -150)

    # best yet
    #kernel = np.array([
    #  [-1, -1, -1],
    #  [-1, 7, -1],
    #  [-1, -1, -1]
    #])
    #img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    #img = cv2.filter2D(img,-1,kernel)
    #img = cv2.bilateralFilter(img, 9, 75, 75)
    #img = cv2.fastNlMeansDenoising(img, None, h=15, templateWindowSize=7, searchWindowSize=21)
    #img = cv2.bilateralFilter(img, 9, 75, 75)
    #img = cv2.addWeighted(img, 1.7, np.zeros(img.shape, img.dtype), 0, -150)

    # ok
    #kernel = np.array([
    #  [-1, 0, 1],
    #  [-2, 0, 2],
    #  [-1, 0, 1]
    #])
    #img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    #img = cv2.filter2D(img,-1,kernel)
    #img = cv2.bilateralFilter(img, 9, 75, 75)
    #img = cv2.fastNlMeansDenoising(img, None, h=15, templateWindowSize=7, searchWindowSize=21)
    #img = cv2.bilateralFilter(img, 9, 75, 75)
    #img = cv2.addWeighted(img, 1.7, np.zeros(img.shape, img.dtype), 0, -150)

    # best thing + emboss
    #kernel = np.array([
    #    [-1, -1, -1],
    #    [-1, 7, -1],
    #    [-1, -1, -1]
    #])
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    #img = cv2.filter2D(img,-1,kernel)
    #img = cv2.bilateralFilter(img, 9, 75, 75)
    #img = cv2.fastNlMeansDenoising(img, None, h=15, templateWindowSize=7, searchWindowSize=21)
    #img = cv2.bilateralFilter(img, 9, 75, 75)
    #img = cv2.addWeighted(img, 1.7, np.zeros(img.shape, img.dtype), 0, -150)
    #kernel = np.array([
   #     [-2, -1, 0],
    #    [-1, 1, 1],
    #    [0, 1, 2]
    #])
    #fil_nl4 = cv2.filter2D(img,-1,kernel)

    
    image_name = Path(image_path).name
    cv2.imwrite(f"./compressed_pictures/{image_name}",img)
    

  0%|          | 0/4118 [00:00<?, ?it/s]

In [3]:
def classes_csv_to_df(file_paths: list, delimiter=";") -> pd.DataFrame:
    df_all = pd.DataFrame()
    for file_path in file_paths:
        df_current = pd.read_csv(file_path, delimiter=delimiter)
        df_all = pd.concat([df_all, df_current])

    df_all = df_all.reset_index(drop=True)
    df_all = df_all.drop_duplicates()
    df_all.drop("Filename", axis=1, inplace=True)
    df_all = remove_unwanted_datapoints(df_all)
    df_all.replace("&Mausohr ", "Mausohr", inplace=True)
    df_all = remove_less_sample_classes(df_all, 60)
    
    #df_all.drop("Species", axis=1, inplace=True)
    return df_all

def categorical_classes(df: pd.DataFrame, column_name: str):
    # df passed as call by reference
    df_copy = df.copy()
    df_copy[column_name] = df_copy[column_name].astype('category')
    return df_copy

def numerical_classes(df: pd.DataFrame, column_name: str) -> tuple[pd.DataFrame, dict[int|str]]:
    df_copy = df.copy()
    df_copy[column_name] = df_copy[column_name].astype('category')
    class_mapping = dict(enumerate(df_copy[column_name].cat.categories))
    df_copy[column_name] = df_copy[column_name].cat.codes
    return df_copy, class_mapping

def encode_classes(df: pd.DataFrame, column_name: str):
    encoded_classes = pd.get_dummies(df[column_name])
    df = df.join(encoded_classes)
    return df

def remove_unwanted_datapoints(df: pd.DataFrame) -> pd.DataFrame:
    ids = set()
    duplicated_ids = set()
    #duplicates = list()
    for row in df.iterrows():
        id = row[1]["ID"]
        if id in ids:
            duplicated_ids.add(id)
            #duplicates.append(row)
        ids.add(id)

    # we drop than since they don't give us information in our trainingsprocess
    #print(len(df[df["Schwarzbild"] == 1]))
    #print(len(df[df["Fledermaus nicht bestimmbar"] == 1]))
    # since this are just 9 we just drop them
    # print(len(duplicates))
    return df[~((df['ID'].isin(duplicated_ids)) |
            (df['Species'].isin(['Fledermaus nicht bestimmbar', 'Schwarzbild'])))]\
            .reset_index(drop=True)

def remove_less_sample_classes(df: pd.DataFrame, min_samples: int) -> pd.DataFrame:
    class_distribution = df['Species'].value_counts()
    valid_classes = class_distribution[class_distribution >= min_samples].index
    return df[df['Species'].isin(valid_classes)]

def class_mapping_to_csv(class_mapping_dict: dict) -> None: 
    with open('data/class_mapping.csv', 'w') as class_mapping_csv:  
        writer = csv.writer(class_mapping_csv)
        for key, value in class_mapping_dict.items():
            writer.writerow([key, value])

def plot_class_distribution(df: pd.DataFrame):
    fig = plt.figure(figsize=(6,6))
    df['Species'].value_counts().plot.bar()
    plt.title('Count Distribution')
    plt.savefig("data/class_distribution")
    plt.close(fig)

df = classes_csv_to_df(["../Auswertung_20220524.csv","../LMU_20180326_class.csv", "../LMU_20180505_classified.csv"])
# RAM is cheaper than salary ;)
#df_categorical = categorical_classes(df, "Species")
df_numerical, class_mapping = numerical_classes(df, "Species")
#df_encoded = encode_classes(df, "Species")
class_mapping_to_csv(class_mapping)
plot_class_distribution(df_numerical)

def get_classes_from_id(id: int, df: pd.DataFrame) -> pd.Series:
    for row in df.iterrows():
        if id == row[1]["ID"]:
            return row[1].drop("ID")

def calc_black_frame(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
    contours,_ = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    x,y,w,h = cv2.boundingRect(cnt)
    return (y, y+h, x, x+w)
    
def load_images_from_folder(folder_path: str, df_categorical=pd.DataFrame(), df_numerical=pd.DataFrame(), df_encoded=pd.DataFrame()) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    if not df_categorical.empty and not df_numerical.empty and not df_encoded.empty:
        raise ValueError("You have to define at least one Dataframe.")
    images_categorical = list()
    images_numerical = list()
    images_encoded = list()
    column_names = ["data", "Species"]
    column_names_encoded = ["data"] + list(df_encoded.columns)[1:]
    for i, filename in enumerate(tqdm(os.listdir(folder_path))):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path):  
            img = cv2.imread(img_path, -1)
            # some images are broken
            if img is not None:
                if i == 0:
                    print("The final image shape/size[hwc] is:",img.shape)
                    # store image shape in file
                    with open("./data/meta.json","w+") as file:
                        file.write(json.dumps({"h":img.shape[0],"w":img.shape[1],"c":img.shape[2]}))
                class_categorical = None
                class_numerical = None
                class_encoded = None
                if not df_categorical.empty:
                    class_categorical = get_classes_from_id(int(filename[:-4]), df_categorical)
                if not df_numerical.empty:
                    class_numerical = get_classes_from_id(int(filename[:-4]), df_numerical)
                if not df_encoded.empty:
                    class_encoded = get_classes_from_id(int(filename[:-4]), df_encoded)
                # need to check, if the class of the image is not null [aka. image would be one of the unwanted datapoints (e.g. class Schwarzbild)]
                if (class_categorical is not None) or (class_numerical is not None) or (class_encoded is not None):
                    if not df_categorical.empty:
                        images_categorical.append([img.flatten(), *class_categorical.values])
                    if not df_numerical.empty:
                        images_numerical.append([img.flatten(), *class_numerical.values])
                    if not df_encoded.empty:
                        images_encoded.append([img.flatten(), *class_encoded.values])

    #print(images_categorical)
    #print(np.array(images_categorical, dtype=object).shape)

    return_dfs = dict()

    if not df_categorical.empty:
        return_dfs["df_categorical"] = pd.DataFrame(np.array(images_categorical, dtype=object), columns=column_names)
    if not df_numerical.empty:
        return_dfs["df_numerical"] = pd.DataFrame(np.array(images_numerical, dtype=object), columns=column_names)
    if not df_encoded.empty:
        return_dfs["df_encoded"] = pd.DataFrame(np.array(images_encoded, dtype=object), columns=column_names_encoded).drop("Species", axis=1)

    return return_dfs

dfs = load_images_from_folder("./compressed_pictures/", df_numerical=df_numerical)
for df_name, df in dfs.items():
    #print(images_categorical["data"].to_numpy().shape)
    if df_name == "df_categorical":
        dfs["df_categorical"].to_pickle("./data/images_df_categorical.pkl")
    if df_name == "df_numerical":
        dfs["df_numerical"].to_pickle("./data/images_df_numerical.pkl")
    if df_name == "df_encoded":
        dfs["df_encoded"].to_pickle("./data/images_df_encoded.pkl")

  0%|          | 0/4118 [00:00<?, ?it/s]

The final image shape/size[hwc] is: (82, 128, 3)


In [4]:
# 1. 1524994153.png -> (10, 75, 15, 116) to (65, 101, 3)
# 2. 1521963463.png -> (10, 75, 16, 116) to (65, 100, 3)